In [1]:
import os
import sys
root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)
!{sys.executable} -m pip install numpy pandas intervaltree matplotlib



You should consider upgrading via the '/Users/alex/dev/mellow-strategy-sdk/examples/.exenv/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
from data import RawData, Pool
raw = RawData(Pool.ETHWBTC)

Done


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from intervaltree import IntervalTree, Interval
from data import RawData, Pool, Frequency
from decimal import Decimal

class LiquidityDistribution:
    def __init__(self):
        self._mints = IntervalTree()
        self._burns = IntervalTree()
    
    def append(self, mints: pd.DataFrame, burns: pd.DataFrame):
        self._mints.update([Interval(mint['tick_lower'], mint['tick_upper'], int(mint['amount'])) for idx, mint in mints.iterrows()]) 
        self._burns.update([Interval(burn['tick_lower'], burn['tick_upper'], int(burn['amount'])) for idx, burn in burns.iterrows()]) 
    
    def at(self, tick: float):
        mint = 0
        [mint := mint + interval.data for interval in self._mints[tick]]
        burn = 0
        [burn := burn + interval.data for interval in self._burns[tick]]
        return mint - burn

class Token:
    def __init__(self, name: str, decimals: int):
        self._name = name
        self._decimals = decimals

    def name(self) -> str:
        return self._name

    def decimals(self) -> int:
        return self._decimals

class PoolData:
    def __init__(self, raw_data: RawData, token0: Token, token1: Token, freq: Frequency):
        self._token0 = token0
        self._token1 = token1
        self._freq = freq
        df = pd.DataFrame()
        decimals_diff = self._token0.decimals() - self._token1.decimals()
        df["c"] = raw_data.swaps["sqrt_price_x96"].transform(lambda x: Decimal(x) * Decimal(x) / Decimal(2 ** 192) * Decimal(10 ** decimals_diff))
        df["c_inv"] = 1 / df["c"]
        self.data = df

# raw.swaps.info()
p = PoolData(raw, Token("WBTC", 8), Token("ETH", 18), Frequency.HOUR)
p.data["c"]
# raw.swaps
# display(p.data)
# display(raw.burns)
# display(raw.swaps)

block_time
2021-05-04 20:22:20    16.03553321631181444171005009
2021-05-04 20:44:52    16.05867170122486852157318891
2021-05-04 21:22:07    16.08165650996577204476841104
2021-05-05 18:16:19    16.50090980789933211284347255
2021-05-05 18:46:25    16.71472018034501878141810103
                                   ...              
2021-06-30 08:32:17    16.33429059502460261883192574
2021-06-30 08:40:25    16.33279307102178441978368637
2021-06-30 08:40:35    16.32942194505976576435251133
2021-06-30 08:41:09    16.32713038243997991418209922
2021-06-30 08:41:26    16.31232120114261190014991779
Name: c, Length: 25290, dtype: object